In [10]:
import requests

In [11]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [12]:
data = requests.get(standings_url)

In [13]:
from bs4 import BeautifulSoup

In [14]:
soup = BeautifulSoup(data.text)

In [ ]:
standings_table = soup.select("table.stats_table")[0]
links = standings_table.find_all("a")
links = [l.get("href") for l in links]
links = [l for l in links if '/squads' in l]
links

In [ ]:
team_urls  = [f"https://fbref.com{l}" for l in links]
data = requests.get(team_urls[0])

import pandas as pd

matches = pd.read_html(data.text, match="Scores & Fixtures")

matches[0]

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all("a")
links = [l.get("href") for l in links]
links = [l for l in links if l and "all_comps/shooting" in l]
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel()
shooting.head()


In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

team_data.head()

In [31]:
years = list(range(2024, 2020, -1))
all_matches = []


[2024, 2023, 2022, 2021]

In [32]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all("a")]
    links = [l for l in links if '/squads' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("Stats", "").replace("-", "")

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in links]
        links = [l for l in links if l and "all_comps/shooting" in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

